In [2]:
import numpy as np
import os
import pandas as pd
import pickle

quad_mmd_output_files = '/media/ak/T7/August11th2022Experiments/QuadMMDOutputFiles'


def pickle_dump_obj_to_filename(destinationPath, symbol, fileName, obj):
    pickle_out_filename = os.path.join(destinationPath, "_".join((symbol, fileName)))
    with open(pickle_out_filename, 'wb') as pickle_out:
        pickle.dump(obj, pickle_out)
    print('saved', pickle_out_filename)


def iterate_and_remove_empty_entries(dictionary):
    clean_dictionary = dictionary.copy()
    for key, value in clean_dictionary.items():
        if isinstance(value, dict):
            clean_dictionary[key] = iterate_and_remove_empty_entries(value)
        elif value is None:
            del clean_dictionary[key]
    return clean_dictionary


def process_dictionary_into_dataframe(clean_dictionary):
    data = []
    columns = ['perm_mmds1', 'chi2_weights', 'sim_mmds', 'sig2', 'Kxy', 'mean', 'var',
               'Kxx', 'Kyy', 'mean_gram', 'var_gram', 'med', 'list_gwidth', 'besti',
               'powers', 'best_ker', 'XZ_test', 'YZ_test']

    for key, val in clean_dictionary.items():
        row = {column: np.nan for column in columns}
        index_values = list(key)
        results = val.get('results', {})
        for result_key, result_val in results.items():
            if result_key in columns:
                if isinstance(result_val, list):
                    row[result_key] = np.median(result_val)
                else:
                    row[result_key] = result_val
        row['index_values'] = tuple(index_values)
        data.append(row)

    df = pd.DataFrame(data)
    df = df.dropna(axis=0, how='any')
    df['index_values'] = df['index_values'].to_numpy()
    df.set_index('index_values', inplace=True)
    return df


if __name__ == "__main__":
    symbol = 'XM1'
    var = 'alpha'
    quadAlpha = os.path.join(quad_mmd_output_files, var)
    files = [f for f in os.listdir(quadAlpha) if str(symbol) in f]
    bar_choice = 'tick'
    varFile = [f for f in files if str(bar_choice) in f][0]
    varFileLoc = os.path.join(quadAlpha, varFile)
    print(varFileLoc)
    pickled_dict = pd.read_pickle(varFileLoc)
    clean_dictionary = iterate_and_remove_empty_entries(pickled_dict)
    df = process_dictionary_into_dataframe(clean_dictionary)
    print(df)


/media/ak/T7/August11th2022Experiments/QuadMMDOutputFiles/alpha/XM1_tick_processedQUADMMDresults_alpha.pickle


RuntimeError: dictionary changed size during iteration

In [ ]:
iterate_and_remove_empty_entries(pd.read_pickle(varFileLoc))